To check this worksheet on Big query (click on below link): 

https://console.cloud.google.com/bigquery?sq=704847300197:813fa6fcbf81419381b96ec5003398c7

Paramétiser les tables : 
- les clients français uniquement (variable NATIONALITE_CLIENT dans la table CLIENTS)
- les achats uniquement s’ils ont été réalisés en France, Belgique, Espagne, Pologne ou Suisse (variable PAYS_ACHAT dans la table VENTES)

In [ ]:
SELECT
  DISTINCT NATIONALITE_CLIENT
FROM
  `big-query-tutorship.Etam.clients`;


    
SELECT
  DISTINCT PAYS_ACHAT
FROM
  `big-query-tutorship.Etam.sales`;

1. Quel est le chiffre d’affaires mensuel des magasins n° 2179 et 2536 ?
(variable PLANT dans la table VENTES = ID magasin)

In [ ]:
SELECT
  SUBSTRING(CAST(DATE_ACHAT AS STRING), 6, 2) AS Mois, 
  ROUND(SUM(CATTC), 2) AS CA_Mensuel
FROM
  `big-query-tutorship.Etam.sales`
WHERE 
  PLANT IN (2197, 2536)
GROUP BY
  SUBSTRING(CAST(DATE_ACHAT AS STRING), 6, 2)
ORDER BY 
  Mois ASC;

2. Quel est l'âge moyen des clients ayant acheté du NIGHTWEAR en janvier 2023 ? Celui de ceux n’en ayant pas acheté sur la même période ? (variable SEGMENT dans table PRODUITS)


In [ ]:
SELECT 
  CASE WHEN p.SEGMENT = 'NIGHTWEAR' THEN p.SEGMENT ELSE 'Others' END AS Nightwear_flag, 
  ROUND(AVG(date_diff(DATE_ACHAT, DATE_DE_NAISSANCE, year)), 2) AS Avg_age
FROM 
  `big-query-tutorship.Etam.sales` s
LEFT JOIN 
  `big-query-tutorship.Etam.clients` c
ON 
  s.RCUCRM = c.RCUCRM
LEFT JOIN 
  `big-query-tutorship.Etam.products` p
ON 
  s.MCVT = p.MCVT
WHERE 
  LEFT(CAST(DATE_ACHAT AS STRING), 7) = '2023-01'
GROUP BY 
  CASE WHEN p.SEGMENT = 'NIGHTWEAR' THEN p.SEGMENT ELSE 'Others' END;

3. Quel est le taux de churn des clients ayant acheté en 2021 ET 2022 ?

(On définit ici le churn comme suit : achat en 2021 ET 2022 mais aucun achat en 2023)

In [ ]:
WITH client_2021 AS 
  (SELECT 
    DISTINCT RCUCRM
  FROM 
    `big-query-tutorship.Etam.sales`
  WHERE LEFT(CAST(DATE_ACHAT AS STRING), 4) = '2021'),

client_2022 AS 
  (SELECT 
    DISTINCT RCUCRM
  FROM 
    `big-query-tutorship.Etam.sales`
  WHERE LEFT(CAST(DATE_ACHAT AS STRING), 4) = '2022'), 

client_2021_2022 AS 
  (SELECT 
    a.*
  FROM 
    client_2021 a
  INNER JOIN 
    client_2022 b
  ON 
    a.RCUCRM = b.RCUCRM
  ), 

client_chrun AS 
  (SELECT 
    RCUCRM 
  FROM 
    client_2021_2022
  WHERE 
    RCUCRM NOT IN (SELECT DISTINCT RCUCRM FROM `big-query-tutorship.Etam.sales` WHERE LEFT(CAST(DATE_ACHAT AS STRING), 4) = '2023')
  )

SELECT 
  ROUND((SELECT COUNT(*) FROM client_chrun) / (SELECT COUNT(*) FROM client_2021_2022), 2)

4. Quelle est la fréquence d'achat 2023 des clients qui étaient omnicanaux en 2023 ? Comment se découpe cette fréquence entre le MAG et le WEB ?

In [ ]:
WITH client_mag AS
  (SELECT 
  DISTINCT RCUCRM
FROM 
  `big-query-tutorship.Etam.sales`
WHERE 
  LEFT(CAST(DATE_ACHAT AS STRING), 4) = '2023'
AND 
  CANAL = 'MAG'), 

client_web AS 
  (SELECT 
  DISTINCT RCUCRM
FROM 
  `big-query-tutorship.Etam.sales`
WHERE 
  LEFT(CAST(DATE_ACHAT AS STRING), 4) = '2023'
AND 
  CANAL = 'WEB'), 

client_omni AS 
  (SELECT 
    a.*
  FROM 
    client_mag a
  INNER JOIN 
    client_web b
  ON 
    a.RCUCRM = b.RCUCRM )

SELECT 
  a.CANAL, 
  ROUND(COUNT(*) / COUNT(DISTINCT a.RCUCRM), 2) AS frequence_achat
FROM 
  `big-query-tutorship.Etam.sales` a 
INNER JOIN 
  client_omni b 
ON 
  a.RCUCRM = b.RCUCRM
WHERE 
  LEFT(CAST(a.DATE_ACHAT AS STRING), 4) = '2023'
GROUP BY
  a.CANAL

5. Quels sont les IC des clients de 2023 selon leur tranche d'âge ?

Définition des IC (indicateurs commerciaux) :
- Panier moyen = somme CA / total achats uniques
- Fréquence d’achat = total achats uniques / total clients uniques
- CA moyen = somme CA / total clients uniques
- Nombre d’articles moyen au panier = total articles / total achats uniques

Vous êtes libre de définir les tranches d’âge qui vous semble les plus cohérentes.

In [ ]:
WITH sales_client AS 
  (SELECT 
  a.*, 
  DATE_DIFF(a.DATE_ACHAT, b.DATE_DE_NAISSANCE, year) AS age
FROM 
  `big-query-tutorship.Etam.sales` a
LEFT JOIN 
  `big-query-tutorship.Etam.clients` b
ON
  a.RCUCRM = b.RCUCRM)

SELECT 
  CASE WHEN age <= 16 THEN 'Enfance' 
    WHEN age <= 32 THEN 'Jeunesse'
    WHEN age <= 49 THEN 'Maturité'
    WHEN age <= 64 THEN 'Pré-retraite'
    ELSE 'Retraite' END AS tranche_age, 
  
  CASE WHEN age <= 16 THEN 1 
    WHEN age <= 32 THEN 2
    WHEN age <= 49 THEN 3
    WHEN age <= 64 THEN 4
    ELSE 5 END AS age_flag,
  
  ROUND(SUM(CATTC) / COUNT(DISTINCT NUM_ACHAT) , 2) AS Panier_moyen,
  ROUND(COUNT(DISTINCT NUM_ACHAT) / COUNT(DISTINCT RCUCRM), 2) AS Frequence_achat,
  ROUND(SUM(CATTC) / COUNT(DISTINCT RCUCRM), 2) AS CA_moyen,
  ROUND(COUNT(*) / COUNT(DISTINCT MCVT), 2) AS Nbr_article_moyen_au_panier

FROM  
  sales_client 
GROUP BY 
  1, 2
ORDER BY age_flag ASC

6. En 2023, quels sont les segments les plus fréquemment cross-sellés avec un achat contenant du NIGHTWEAR ? 

In [ ]:
WITH nightwear_order AS    
  (SELECT 
    DISTINCT s.NUM_ACHAT, 
    SEGMENT
  FROM 
    `big-query-tutorship.Etam.sales` s 
  INNER JOIN  
    `big-query-tutorship.Etam.products` p
  ON 
    s.MCVT = p.MCVT
  WHERE 
    p.SEGMENT = 'NIGHTWEAR'), 

others_order AS    
  (SELECT 
      DISTINCT s.NUM_ACHAT, 
      SEGMENT
    FROM 
      `big-query-tutorship.Etam.sales` s 
    INNER JOIN  
      `big-query-tutorship.Etam.products` p
    ON 
      s.MCVT = p.MCVT
    WHERE 
      p.SEGMENT <> 'NIGHTWEAR'),

cross_selling_orders AS    
  (SELECT 
  a.*, 
  b.SEGMENT AS cross_selling_segment
  FROM 
    nightwear_order a  
  INNER JOIN  
    others_order b 
  ON a.NUM_ACHAT = b.NUM_ACHAT)

SELECT 
  SEGMENT, 
  cross_selling_segment, 
  COUNT(NUM_ACHAT) AS frequency
FROM 
  cross_selling_orders
GROUP BY 
  1, 2
ORDER BY 3 DESC

Combien d’achats ne contenaient QUE du NIGHTWEAR ?

In [ ]:
WITH nightwear_order AS    
  (SELECT 
    DISTINCT s.NUM_ACHAT, 
    SEGMENT
  FROM 
    `big-query-tutorship.Etam.sales` s 
  INNER JOIN  
    `big-query-tutorship.Etam.products` p
  ON 
    s.MCVT = p.MCVT
  WHERE 
    p.SEGMENT = 'NIGHTWEAR'), 

others_order AS    
  (SELECT 
      DISTINCT s.NUM_ACHAT, 
      SEGMENT
    FROM 
      `big-query-tutorship.Etam.sales` s 
    INNER JOIN  
      `big-query-tutorship.Etam.products` p
    ON 
      s.MCVT = p.MCVT
    WHERE 
      p.SEGMENT <> 'NIGHTWEAR')

SELECT 
  COUNT(DISTINCT a.NUM_ACHAT) AS ONLY_BUY_NIGHTWEAR
FROM 
  nightwear_order a
LEFT JOIN 
  others_order b
ON 
  a.NUM_ACHAT = b.NUM_ACHAT  AND a.SEGMENT = b.SEGMENT


7. Quel est le panier moyen des clients ayant réalisé 1 achat en 2023 vs 2+ achats ?
Panier moyen = somme CA / total achats uniques

In [ ]:
WITH nbr_achat_client AS   
  (SELECT 
    RCUCRM, 
    COUNT(DISTINCT NUM_ACHAT) AS nbr_achat 
  FROM  
    `big-query-tutorship.Etam.sales`
  GROUP BY 
    RCUCRM), 

client_label AS    
  (SELECT 
    *, 
    CASE WHEN nbr_achat = 1 THEN "1 achat" ELSE "2+ achats" END AS label 
  FROM   
    nbr_achat_client)

SELECT 
  b.label, 
  ROUND(SUM(a.CATTC) / COUNT(DISTINCT a.NUM_ACHAT), 2) AS Panier_moyen 
FROM 
  `big-query-tutorship.Etam.sales` a
INNER JOIN  
  client_label b
ON 
  a.RCUCRM = b.RCUCRM
GROUP BY 
  b.label
ORDER BY 1 ASC
